Consider the n-dimensional linear system of ordinary differential equations:
$$x'(t)=A(t)x(t)$$
where in addition, the matrix $A(t)$ is periodic with period $T$, i.e. $A(t+T)=A(t)$ for all real values of $t$.
Let $Φ(t)$ denote the principal fundamental matrix solution such that the columns
of $Φ(t)$ are linearly independent, and $Φ(0)=I$. Then, Floquet's theorem decomposes the principal
fundamental matrix as the product of a periodic matrix $P(t)$ with period $T$ and an exponential matrix $e^{tB}$.



In [1]:
using Plots, ApproxFun

In [2]:
function Base.expm(A::Matrix{T}, f::Fun) where {T<:Number}
    D,V = eig(A)
    fC = D == real(D) ? fill(f,n) : fill(f+0im,n)
    for i = 1:length(D)
        fC[i] = Fun(t->exp(D[i]*fC[i](t)),domain(f))
    end
    V*diagm(fC)*inv(V)
end;

Consider the system of coupled harmonic oscillators with periodic parameterized excitation:
     $$x'' + (1+a \cos(2t))x = x-y$$
     $$y'' + (1+a \cos(2t))y = y-x$$
We calculate the Principal Fundamental Matrix $Φ$.

In [3]:
T = π; a=0.15
t = Fun(identity,0..T)
d=domain(t)
D=Derivative(d)
B=ldirichlet(d)
n=4
A=[ Operator(diagm(fill(ldirichlet(d),n)));
    D             -I  0I            0I;
   (2+a*cos(2t))   D  -I            0I;
   0I             0I   D            -I;
   -I             0I  (2+a*cos(2t))  D]

# we need eye(n) for the initial condition and zeros(n,n) for the
# right-hand side of the ODE

Φ = A\eye(2n,n);  

It can be further expressed as $Φ(t) = P(t) e^{tB}$ where $P(t)$ is a periodic matrix, and $B$ is constant.
Using the values of $Φ(T)$:

In [4]:
ΦT = Φ(T)

4×4 Array{Float64,2}:
 -0.170879  -0.148885  -0.836059   0.265569
  0.732284  -0.170879  -0.612945  -0.836059
 -0.836059   0.265569  -0.170879  -0.148885
 -0.612945  -0.836059   0.732284  -0.170879

we calculate the matrix $B$ and the Floquet exponents and multipliers.

In [5]:
B = logm(ΦT)/T
D,V = eig(B)

println("Floquet Exponents = \n")
[println(D[i]) for i=1:n]
println("\nFloquet Multipliers = \n")
[println(exp(D[i]*T)) for i=1:n];

Floquet Exponents = 

3.885780586188048e-16 - 0.2683546905319055im
2.0995575070181793e-16 + 0.26835469053190514im
-0.037475319732289195 + 0.9999999999999997im
0.037475319732289236 + 0.9999999999999999im

Floquet Multipliers = 

0.6651802570066608 - 0.7466828146465907im
0.6651802570066614 + 0.7466828146465896im
-0.8889340869219555 + 8.98395100960111e-16im
-1.1249427991479368 + 6.373407186201634e-16im


We can build the matrix exponential $e^{-tB}$ and use it to find the periodic matrix $P(t)$.
Periodicity is numerically confirmed by `Fun` instantiation with low number of coefficients.

In [6]:
PI = Φ*expm(B,-t)
P = Fun(t->PI(t),PeriodicInterval(d))
P|>ncoefficients

223

With the matrix B and the periodic matrix P(t), we can construct 
the solution with any initial conditions for as long as we want!

In [7]:
t = Fun(identity,0..10T)
d = domain(t)
x0 = rand(n)

PI = Fun(t->P(t),d)
xsol = real(PI*expm(B,t)*x0)

plot(xsol)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -4 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 -0.75 
 
 
 -0.50 
 
 
 -0.25 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75 
 
<polyline clip-path="url(#clip7002)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 46.1926,200.096 95.0558,200.096 124.421,200.096 149.248,200.096 176.218,200.096 201.081,200.096 227.738,200.096 258.622,200.096 272.345,200.096 286.068,200.096 
 292.085,200.096 298.102,200.096 301.11,200.096 304.119,200.096 305.623,200.096 307.127,200.096 307.879,200.096 308.632,200.096 309.384,200.096 310.136,200.096 
 310.546,200.096 310.956,200.096 311.161,200.096 311.366,200.096 311.571,42.9758 311.776,42.7857 311.981,42.5994 312.186,42.4169 312.391,42.2382 312.596,42.0633 
 313.006,41.7251 313.416,41.4023 315.056,40.2653 316.696,39.3766 319.976,38.3492 323.256,38.3241 326.536,39.2962 329.816,41.2514 333.096,44.1667 336.376,48.011 
 339.676,52.7761 342.975,58.3954 346.275,64.8155 349.574,71.9777 352.874,79.8185 356.173,88.2704 359.473,97.2629 362.773,106.723 377.365,152.266 391.957,199.203 
 398.899,220.352 405.841,240.014 412.783,257.747 419.725,273.186 423.334,280.209 426.943,286.505 430.553,292.049 434.162,296.825 437.771,300.824 441.38,304.042 
 444.989,306.485 448.598,308.165 454.674,309.323 460.75,308.509 466.826,305.908 472.903,301.749 480.321,294.945 487.739,286.735 495.157,277.655 502.576,268.213 
 509.549,259.41 516.522,250.988 523.496,243.165 530.469,236.066 541.641,226.254 552.813,218.046 563.985,210.691 575.157,203.187 576.876,201.951 578.596,200.683 
 580.315,199.379 
 "/>
<polyline clip-path="url(#clip7002)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 46.1926,200.096 95.0558,200.096 124.421,200.096 149.248,200.096 176.218,200.096 201.081,200.096 227.738,200.096 258.622,200.096 272.345,200.096 286.068,200.096 
 292.085,200.096 298.102,200.096 301.11,200.096 304.119,200.096 307.127,200.096 310.136,200.096 310.546,200.096 310.956,200.096 311.161,200.096 311.366,200.096 
 311.571,149.406 311.776,150.411 311.981,151.418 312.186,152.426 312.391,153.436 312.596,154.447 313.006,156.472 313.416,158.503 315.056,166.668 316.696,174.888 
 319.976,191.416 323.256,207.935 329.816,240.345 336.376,270.977 339.676,285.385 342.975,298.973 346.275,311.639 349.574,323.301 352.874,333.885 356.173,343.336 
 359.473,351.611 362.773,358.681 366.421,365.076 370.069,369.973 373.717,373.386 377.365,375.338 381.013,375.869 384.661,375.027 388.309,372.869 391.957,369.461 
 395.428,365.122 398.899,359.78 402.37,353.504 405.841,346.365 409.312,338.438 412.783,329.8 416.254,320.53 419.725,310.711 434.162,265.895 448.598,219.638 
 454.674,201.389 460.75,184.575 466.826,169.622 472.903,156.882 476.612,150.306 480.321,144.688 484.03,140.049 487.739,136.392 491.448,133.702 495.157,131.949 
 498.867,131.083 502.576,131.043 509.549,132.926 516.522,136.842 523.496,142.099 530.469,147.969 541.641,156.909 552.813,163.145 558.399,164.678 563.985,164.97 
 569.571,163.963 575.157,161.669 576.876,160.713 578.596,159.647 580.315,158.473 
 "/>
<polyline clip-path="url(#clip7002)" style="stroke:#3da44d; stroke-width:1; stroke-opacity:1; fill:none" points="
 46.1926,200.096 95.0558,200.096 124.421,200.096 149.248,200.096 176.218,200.096 201.081,200.096 227.738,200.096 258.622,200.096 272.345,200.096 286.068,200.096 
 292.085,200.096 298.102,200.096 301.11,200.096 304.119,200.096 305.623,200.096 307.127,200.096 307.879,200.096 308.632,200.096 309.384,200.096 310.136,200.096 
 310.546,200.096 310.956,200.096 311.161,200.096 311.366,200.096 311.571,127.46 311.776,127.438 311.981,127.417 312.186,127.395 312.391,127.373 312.596,127.352 
 313.006,127.308 313.416,127.264 315.056,127.089 316.696,126.91 319.976,126.543 323.256,126.16 329.816,125.358 336.376,124.551 349.574,123.34 362.773,123.785 
 370.069,125.34 377.365,128.204 384.661,132.66 391.957,138.958 395.428,142.659